In [1]:
import pandas as pd
import random
import json
from scipy import stats
import statsmodels.api as sm
from bokeh.io import output_notebook, output_file , show
from bokeh.plotting import figure
from bokeh.models import Range1d

from bokeh.models import ColumnDataSource, FactorRange
from bokeh.models import Label, LabelSet, Range1d
from bokeh.transform import dodge

from bokeh.transform import factor_cmap
from bokeh.palettes import Category10, Greys

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

output_notebook()
my_pallete = Greys[9]

Loading BokehJS ...

In [4]:
df = pd.read_parquet("LT_Dataset_full_u.parquet")
print(df.shape)
df.head()

(164275, 14)


,_id,author_affiliation,author_name,para_text,site_url,source,title,year,author_affiliation_short,ae,pc,manifesto,year_rec,parliamentary
0,1333c43a-f51b-4ba6-bf03-b79d5dd3be6e,TS-LKD,G. Landsbergis,Šeštadienį Vilniuje vykstančiame Tėvynė...,https://tsajunga.lt/aktualijos/ts-lkd-taryba-p...,tsajunga.lt/,TS-LKD Taryba patvirtino naujas programines ga...,2016,TS-LKD,True,True,0,2016,1
1,658d74f5-a958-41f2-9f0c-81c492e615a4,TS-LKD,G. Landsbergis,„Artėjančius 2016 metų Seimo rinkimus pasitink...,https://tsajunga.lt/aktualijos/ts-lkd-taryba-p...,tsajunga.lt/,TS-LKD Taryba patvirtino naujas programines ga...,2016,TS-LKD,True,True,0,2016,1
2,4cfaa0d4-71c9-4618-b0e2-25cadeebfc74,TS-LKD,G. Landsbergis,TS-LKD programinėse gairėse „Teisingesnė ir pa...,https://tsajunga.lt/aktualijos/ts-lkd-taryba-p...,tsajunga.lt/,TS-LKD Taryba patvirtino naujas programines ga...,2016,TS-LKD,False,True,0,2016,1
3,3a106bd2-82b1-45dd-8a31-eccc379a0e7b,TS-LKD,G. Landsbergis,„Nomenklatūrinei kairei kartu su populistinėms...,https://tsajunga.lt/aktualijos/ts-lkd-taryba-p...,tsajunga.lt/,TS-LKD Taryba patvirtino naujas programines ga...,2016,TS-LKD,True,True,0,2016,1
4,2c53d4a7-9c2e-41b6-b2e3-3e198f68fdb8,TS-LKD,V. Juozapaitis,Vakar Seime įvyko šešėlinės Vyriausybės...,https://tsajunga.lt/aktualijos/v-juozapaitis-l...,tsajunga.lt/,Lietuvoje stinga principinio kultūros politiko...,2016,TS-LKD,True,False,0,2016,1


## H1 Populism Over Time

### H1 a - Manifestos

In [5]:
df_h1_1 = df.groupby(['year_rec' , "manifesto"])["ae", "pc"].mean()
df_h1_2 = pd.DataFrame()
df_h1_2["year"] = [i[0] for i in list(df_h1_1.index)]
df_h1_2["manifesto"] = [i[1] for i in list(df_h1_1.index)]
df_h1_2["ae"] = list(df_h1_1["ae"])
df_h1_2["pc"] = list(df_h1_1["pc"])

df_h1_2.head()

<ipython-input-5-eaed256512d4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h1_1 = df.groupby(['year_rec' , "manifesto"])["ae", "pc"].mean()


,year,manifesto,ae,pc
0,1992,1,0.103960,0.166337
1,1996,1,0.080264,0.136449
2,2000,1,0.068731,0.136455
3,2004,0,0.800000,0.200000
4,2004,1,0.067651,0.156265


In [6]:
df_h1_3 = df_h1_2.loc[df_h1_2["manifesto"]==1]
df_h1_3.head(20)

,year,manifesto,ae,pc
0,1992,1,0.103960,0.166337
1,1996,1,0.080264,0.136449
2,2000,1,0.068731,0.136455
4,2004,1,0.067651,0.156265
6,2008,1,0.101185,0.192252
8,2012,1,0.087531,0.164577
10,2016,1,0.087725,0.168413
12,2020,1,0.077687,0.186008


In [7]:
#output_file("Fig1_Populism_Manifestos.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.6), title = "Populist Content in Election Manifestos: All parties" )
p.line(list(df_h1_3["year"]), list(df_h1_3["ae"]), color="black", alpha=1, legend="AE", line_width=4)
p.line(list(df_h1_3["year"]), list(df_h1_3["pc"]), color="grey", alpha=1, legend="PC", line_width=4)
show(p)

### H1 b - Media

In [8]:
df_h1_4 = df.groupby(['year' , "manifesto"])["ae", "pc"].mean()
df_h1_5 = pd.DataFrame()
df_h1_5["year"] = [i[0] for i in list(df_h1_4.index)]
df_h1_5["manifesto"] = [i[1] for i in list(df_h1_4.index)]
df_h1_5["ae"] = list(df_h1_4["ae"])
df_h1_5["pc"] = list(df_h1_4["pc"])

df_h1_5.head()

<ipython-input-8-3034b5e6c1da>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h1_4 = df.groupby(['year' , "manifesto"])["ae", "pc"].mean()


,year,manifesto,ae,pc
0,1992,1,0.103960,0.166337
1,1996,1,0.080264,0.136449
2,2000,1,0.068731,0.136455
3,2003,0,1.000000,1.000000
4,2004,0,0.750000,0.000000


In [9]:
df_h1_6 = df_h1_5.loc[df_h1_5["manifesto"]==0]
df_h1_6 = df_h1_6.loc[df_h1_6["year"]>2005 ]
df_h1_6.head()

,year,manifesto,ae,pc
7,2006,0,0.750000,0.416667
8,2007,0,0.702128,0.553191
9,2008,0,0.800000,0.552941
11,2009,0,0.544693,0.410615
12,2010,0,0.515312,0.352341


In [10]:
#output_file("Fig2_Populism_Media.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 1), title = "Populist Content in Media: All parties")
p.line(list(df_h1_6["year"]), list(df_h1_6["ae"]), color="black", alpha=1, legend="AE", line_width=4)
p.line(list(df_h1_6["year"]), list(df_h1_6["pc"]), color="grey", alpha=1, legend="PC", line_width=4)
show(p)

## Combo

In [11]:
df_h1_7 = df_h1_2.loc[df_h1_2["manifesto"]==0]
df_h1_7 = df_h1_7.loc[df_h1_7["year"]>2004 ]
df_h1_7.head()

,year,manifesto,ae,pc
5,2008,0,0.765517,0.537931
7,2012,0,0.509404,0.362122
9,2016,0,0.401706,0.327216
11,2020,0,0.471418,0.337464


In [12]:
#output_file("Fig3_Populism_combo_v1.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", title = "Share of Populist Content: by Source")
p.line(list(df_h1_3["year"]), list(df_h1_3["ae"]), color="black", alpha=1, line_width=4)
p.line(list(df_h1_3["year"]), list(df_h1_3["pc"]), color="grey", alpha=1, line_width=4)
p.circle(list(df_h1_3["year"]), list(df_h1_3["ae"]), color="black", alpha=1, legend="Manifesto AE", size =8)
p.circle(list(df_h1_3["year"]), list(df_h1_3["pc"]), color="grey", alpha=1, legend="Manifesto PC", size =8)

p.square(list(df_h1_6["year"]), list(df_h1_6["ae"]), color="black", alpha=1, legend="Media AE", size =8)
p.square(list(df_h1_6["year"]), list(df_h1_6["pc"]), color="grey", alpha=1, legend="Media PC", size =8)
p.line(list(df_h1_6["year"]), list(df_h1_6["ae"]), color="black", alpha=1, line_width=4)
p.line(list(df_h1_6["year"]), list(df_h1_6["pc"]), color="grey", alpha=1, line_width=4)
p.legend.location="top_left"
show(p)

In [13]:
#output_file("Fig4_Populism_combo_v2.html")
tools = "hover"
year_ae = [int(i)-0.25 for i in list(df_h1_3["year"])]
year_pc = [int(i)+0.25 for i in list(df_h1_3["year"])]
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 1), title = "Share of Populist Content: by Source")
p.vbar(x=year_ae, top=list(df_h1_3["ae"]), color="black", width=0.5, legend="Manifesto AE", )
p.vbar(x=year_pc, top=list(df_h1_3["pc"]), color="grey", width=0.5, legend="Manifesto PC")

p.circle(list(df_h1_6["year"]), list(df_h1_6["ae"]), color="black", alpha=1, legend="Media AE", size =8)
p.circle(list(df_h1_6["year"]), list(df_h1_6["pc"]), color="grey", alpha=1, legend="Media PC", size =8)
p.line(list(df_h1_6["year"]), list(df_h1_6["ae"]), color="black", alpha=1, line_width=4)
p.line(list(df_h1_6["year"]), list(df_h1_6["pc"]), color="grey", alpha=1, line_width=4)
p.legend.location="top_left"
show(p)

# H3

In [14]:
df_h3_1 = df.loc[df["manifesto"]==1 ]
df_h3_2 = df_h3_1.groupby(['year' , "parliamentary"])["ae", "pc"].mean()

df_h3_2

<ipython-input-14-c80fad7e0967>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h3_2 = df_h3_1.groupby(['year' , "parliamentary"])["ae", "pc"].mean()


ae        pc
year parliamentary                    
1992 0              0.103960  0.166337
1996 0              0.078125  0.132812
     1              0.080402  0.136683
2000 0              0.103067  0.181595
     1              0.059867  0.124802
2004 0              0.104592  0.172194
     1              0.045627  0.146768
2008 0              0.094340  0.188679
     1              0.102008  0.192681
2012 0              0.137701  0.250000
     1              0.077219  0.147018
2016 0              0.096491  0.162281
     1              0.087082  0.168862
2020 0              0.120598  0.215582
     1              0.062721  0.175693

In [15]:
df_h3_3 = pd.DataFrame()
df_h3_3["year"] = [i[0] for i in df_h3_2.index]
df_h3_3["parliamentary"] = [i[1] for i in df_h3_2.index]
df_h3_3["ae"] = list(df_h3_2["ae"])
df_h3_3["pc"] = list(df_h3_2["pc"])

In [16]:
df_h3_4 = df_h3_3.loc[(df_h3_3["year"]>1992)&(df_h3_3["parliamentary"]==1)]

#output_file("Fig5_Populism_ParliamentaryOnly.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.6), title = "Populist Content in Manifestos: Parliamentary parties")
p.line(list(df_h3_4["year"]), list(df_h3_4["ae"]), color="black", alpha=1, legend="AE", line_width = 4)
p.line(list(df_h3_4["year"]), list(df_h3_4["pc"]), color="grey", alpha=1, legend="PC", line_width=4)
show(p)

In [17]:
df_h3_5 = df_h3_3.loc[(df_h3_3["year"]>1992)&(df_h3_3["parliamentary"]==0)]

#output_file("Fig6_Populism_NonParliamentaryOnly.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.6), title = "Populist Content in Manifestos: Non-parliamentary parties")
p.line(list(df_h3_5["year"]), list(df_h3_5["ae"]), color="black", alpha=1, legend="AE", line_width=4)
p.line(list(df_h3_5["year"]), list(df_h3_5["pc"]), color="grey", alpha=1, legend="PC", line_width=4)
show(p)

In [18]:
#output_file("Fig7_Populism_ParlNonParlCombo.html")
tools = "hover"
year_ae = [int(i)-0.25 for i in list(df_h3_4["year"])]
year_pc = [int(i)+0.25 for i in list(df_h3_4["year"])]

p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.6), title = "Populist Content in Manifestos by party type")
p.vbar(x=year_ae, top=list(df_h3_4["ae"]), color="black", width=0.5, legend="Parliamentary AE", )
p.vbar(x=year_pc, top=list(df_h3_4["pc"]), color="grey", width=0.5, legend="Parliamentary PC")

p.circle(list(df_h3_5["year"]), list(df_h3_5["ae"]), color="black", alpha=1, legend="Non-Parliamentary AE", size=8)
p.circle(list(df_h3_5["year"]), list(df_h3_5["pc"]), color="grey", alpha=1, legend="Non-Parliamentary PC", size=8)
p.line(list(df_h3_5["year"]), list(df_h3_5["ae"]), color="black", alpha=1, line_width=4)
p.line(list(df_h3_5["year"]), list(df_h3_5["pc"]), color="grey", alpha=1, line_width=4)
p.legend.location="top_left"
show(p)

# Parliamentary paries' PC and AE over time

In [19]:
df_h4_1 = df.loc[(df["manifesto"] == 1) & df["parliamentary"]==1]
print(df_h4_1.shape)
df_h4_1.head()

(24786, 14)


,_id,author_affiliation,author_name,para_text,site_url,source,title,year,author_affiliation_short,ae,pc,manifesto,year_rec,parliamentary
133303,5f91c7844307556d58e8ff02,LKDP,LKDP,TARNAUTI LIETUVAI,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1
133304,5f91c7874307556d58e8ff03,LKDP,LKDP,LIETUVOS KRIKŠČIONIŲ DEMOKRATŲ PARTIJOS RINKIM...,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1
133305,5f91c8224307556ebcd00599,LKDP,LKDP,TARNAUTI LIETUVAI,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1
133306,5f91c8254307556ebcd0059a,LKDP,LKDP,LIETUVOS KRIKŠČIONIŲ DEMOKRATŲ PARTIJOS RINKIM...,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1
133307,5f91c8274307556ebcd0059b,LKDP,LKDP,2000,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1


In [20]:
df_h4_2 = df_h4_1.groupby(["year" , "author_affiliation_short"])["ae", "pc"].mean()
print(df_h4_2.shape)
df_h4_2.head()

(53, 2)


<ipython-input-20-e8f6ad966bd6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h4_2 = df_h4_1.groupby(["year" , "author_affiliation_short"])["ae", "pc"].mean()


ae        pc
year author_affiliation_short                    
1996 LCS                       0.099099  0.219219
     LDDP                      0.082192  0.123288
     LKDP                      0.040462  0.098266
     LSDP                      0.103774  0.150943
     TS-LKD                    0.077739  0.088339

In [21]:
df_h4_3 = pd.DataFrame()
df_h4_3["year"] = [i[0] for i in list(df_h4_2.index) ]
df_h4_3["party"] = [i[1] for i in list(df_h4_2.index) ]
df_h4_3["ae"] = list(df_h4_2["ae"])
df_h4_3["pc"] = list(df_h4_2["pc"])

print(df_h4_3.shape)
df_h4_3.head()

(53, 4)


,year,party,ae,pc
0,1996,LCS,0.099099,0.219219
1,1996,LDDP,0.082192,0.123288
2,1996,LKDP,0.040462,0.098266
3,1996,LSDP,0.103774,0.150943
4,1996,TS-LKD,0.077739,0.088339


In [22]:
data_ae = {}

for index, row in df_h4_3.iterrows():
    if row["year"] not in data_ae.keys() :
        data_ae[ row["year"]] = {}
    data_ae[ row["year"]][row["party"]] = row["ae"]
    
data_ae2 = []
for key, value in data_ae.items():
    value.update({"year" : key})
    data_ae2.append(value)
    
df_h4_4ae = pd.DataFrame(data_ae)
print(df_h4_4ae.shape)
df_h4_4ae.head()

(21, 7)


,1996,2000,2004,2008,2012,2016,2020
LCS,0.099099,0.029268,NaN,NaN,NaN,NaN,NaN
LDDP,0.082192,NaN,NaN,NaN,NaN,NaN,NaN
LKDP,0.040462,0.036649,0.000000,NaN,NaN,NaN,NaN
LSDP,0.103774,0.086876,0.022277,0.112128,0.041916,0.021097,0.132931
TS-LKD,0.077739,0.069307,0.062016,0.128571,0.110217,0.137339,0.060541


In [23]:
df_h4_4ae2= df_h4_4ae.T
df_h4_4ae2.head(10)

,LCS,LDDP,LKDP,LSDP,TS-LKD,year,LLRA,LRLS,taut,CP,...,DP,LIC,NaujSaj,TT,DK,KSS,LSDDP,LZP,Laisve_Teisingumas,Lietuvos_Sarasas
1996,0.099099,0.082192,0.040462,0.103774,0.077739,1996.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,0.029268,NaN,0.036649,0.086876,0.069307,2000.0,0.046729,0.037037,0.340909,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,NaN,NaN,0.000000,0.022277,0.062016,2004.0,0.140000,NaN,0.076923,0.0625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,0.112128,0.128571,2008.0,0.128205,NaN,NaN,NaN,...,0.022814,0.036789,0.0,0.082437,NaN,NaN,NaN,NaN,NaN,NaN
2012,NaN,NaN,NaN,0.041916,0.110217,2012.0,0.159574,0.022792,NaN,NaN,...,0.038911,0.000000,NaN,0.109804,NaN,NaN,NaN,NaN,NaN,NaN
2016,NaN,NaN,NaN,0.021097,0.137339,2016.0,0.221239,0.066845,NaN,NaN,...,0.025682,NaN,NaN,0.026423,0.169014,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,0.132931,0.060541,2020.0,0.170732,0.048565,NaN,NaN,...,0.030349,NaN,NaN,NaN,NaN,0.04902,0.051163,0.016529,0.056738,0.18617


In [24]:
parties = ["DP", "LLRA",  "LSDP", "LVZS" , 'TS-LKD', "TT"]

df_h4_4ae2 = df_h4_4ae2[parties]

#output_file("Fig8_SelectedPartiesAE.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.9), title="Parliamentary Party AE over Time by Party")

for index, col in enumerate(list(df_h4_4ae2.columns)):
    if col != "year":
        pass
        #p.line(list(df_h4_4ae2.index), list(df_h4_4ae2[col]), color=my_pallete[index], alpha=1, legend=col, line_width=4)

p.line( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[0]]), color=my_pallete[0], alpha=1, legend=parties[0], line_width=2)
p.circle( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[0]]), color=my_pallete[0], alpha=1, legend=parties[0], line_width=2, size=10)

p.line( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[1]]), color=my_pallete[2], alpha=1, legend=parties[1], line_width=2)
p.triangle( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[1]]), color=my_pallete[2], alpha=1, legend=parties[1], line_width=2, size=10)
        
p.line( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[2]]), color=my_pallete[3], alpha=1, legend=parties[2], line_width=2)
p.diamond( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[2]]), color=my_pallete[3], alpha=1, legend=parties[2], line_width=2, size=10)

p.line( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[3]]), color=my_pallete[4], alpha=1, legend=parties[3], line_width=2)
p.square( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[3]]), color=my_pallete[4], alpha=1, legend=parties[3], line_width=2, size=10)

p.line( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[4]]), color=my_pallete[5], alpha=1, legend=parties[4], line_width=2)
p.inverted_triangle( list(df_h4_4ae2.index), list(df_h4_4ae2[parties[4]]), color=my_pallete[5], alpha=1, legend=parties[4], line_width=2, size=10)

p.legend.location="top_center"
p.legend.orientation = "horizontal"
p.legend.label_text_font_size = "6pt"
show(p)

In [25]:
data_pc = {}

for index, row in df_h4_3.iterrows():
    if row["year"] not in data_pc.keys() :
        data_pc[ row["year"]] = {}
    data_pc[ row["year"]][row["party"]] = row["pc"]
    
data_pc2 = []
for key, value in data_pc.items():
    value.update({"year" : key})
    data_pc2.append(value)
    
df_h4_4pc = pd.DataFrame(data_pc)
print(df_h4_4pc.shape)
df_h4_4pc.head()

df_h4_4pc2= df_h4_4pc.T
df_h4_4pc2.head()

df_h4_4pc2 = df_h4_4pc2[["DP", "LLRA",  "LSDP", "LVZS" , 'TS-LKD', "TT"]]

output_file("Fig9_SelectedPartiesPC.html")
tools = "hover"
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 0.9), title="Parliamentary Party PC over Time by Party")

for index, col in enumerate(list(df_h4_4pc2.columns)):
    if col != "year":
        #p.line(list(df_h4_4pc2.index), list(df_h4_4pc2[col]), color=my_pallete[index], alpha=1, legend=col, line_width=4)
        pass

p.line( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[0]]), color=my_pallete[0], alpha=1, legend=parties[0], line_width=2)
p.circle( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[0]]), color=my_pallete[0], alpha=1, legend=parties[0], line_width=2, size=10)

p.line( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[1]]), color=my_pallete[2], alpha=1, legend=parties[1], line_width=2)
p.triangle( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[1]]), color=my_pallete[2], alpha=1, legend=parties[1], line_width=2, size=10)
        
p.line( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[2]]), color=my_pallete[3], alpha=1, legend=parties[2], line_width=2)
p.diamond( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[2]]), color=my_pallete[3], alpha=1, legend=parties[2], line_width=2, size=10)

p.line( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[3]]), color=my_pallete[4], alpha=1, legend=parties[3], line_width=2)
p.square( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[3]]), color=my_pallete[4], alpha=1, legend=parties[3], line_width=2, size=10)

p.line( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[4]]), color=my_pallete[5], alpha=1, legend=parties[4], line_width=2)
p.inverted_triangle( list(df_h4_4pc2.index), list(df_h4_4pc2[parties[4]]), color=my_pallete[5], alpha=1, legend=parties[4], line_width=2, size=10)
    
#p.line(list(df_h3_5["year"]), list(df_h3_5["pc"]), color="maroon", alpha=1, legend="PC")
p.legend.location="top_center"
p.legend.orientation = "horizontal"
p.legend.label_text_font_size = "6pt"
show(p)

(21, 7)


# H5

In [26]:
df_h5_1 = df #df.loc[(df["parliamentary"]==1) ]

df_h5_2 = df_h5_1.groupby([ "manifesto", "author_affiliation_short"])["ae", "pc"].mean()
print(df_h5_2.shape)
df_h5_2.head()

(66, 2)


<ipython-input-26-4dc93cbb4625>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_h5_2 = df_h5_1.groupby([ "manifesto", "author_affiliation_short"])["ae", "pc"].mean()


ae        pc
manifesto author_affiliation_short                    
0         CP                        0.531646  0.430380
          DK                        0.630792  0.385650
          DP                        0.365633  0.327368
          Front                     0.695652  0.405797
          LIC                       0.441815  0.327416

In [27]:
df_h5_3 = pd.DataFrame()
df_h5_3["manifesto"] = [i[0] for i in list(df_h5_2.index)]
df_h5_3["party"]=[i[1] for i in list(df_h5_2.index)]
df_h5_3["ae"] = list(df_h5_2["ae"])
df_h5_3["pc"]=list(df_h5_2["pc"])

print(df_h5_3.shape)
df_h5_3.head()

(66, 4)


,manifesto,party,ae,pc
0,0,CP,0.531646,0.430380
1,0,DK,0.630792,0.385650
2,0,DP,0.365633,0.327368
3,0,Front,0.695652,0.405797
4,0,LIC,0.441815,0.327416


In [28]:
df_h5_4 = df_h5_3.fillna(0)

df_h5_5_dict = {}
for index, row in df_h5_4.iterrows():
    if row["party"] not in df_h5_5_dict.keys():
        df_h5_5_dict[row["party"]] = {"party" : row["party"]}
    if row["manifesto"] == 0:
        df_h5_5_dict[row["party"]]["media_ae"] = row["ae"]
        df_h5_5_dict[row["party"]]["media_pc"] = row["pc"]
    else:
        df_h5_5_dict[row["party"]]["manifesto_ae"] = row["ae"]
        df_h5_5_dict[row["party"]]["manifesto_pc"] = row["pc"]
        
df_h5_5 = pd.DataFrame(list(df_h5_5_dict.values()))
print(df_h5_5.shape)
df_h5_5.head()

(41, 5)


,party,media_ae,media_pc,manifesto_ae,manifesto_pc
0,CP,0.531646,0.430380,0.130435,0.304348
1,DK,0.630792,0.385650,0.232804,0.278660
2,DP,0.365633,0.327368,0.033469,0.115332
3,Front,0.695652,0.405797,0.571429,0.428571
4,LIC,0.441815,0.327416,0.045375,0.134380


In [29]:
list_1 = [i-0.15 for i in list(df_h5_5.index)]
list_2 = [i+0.15 for i in list(df_h5_5.index)]

#output_file("Fig10_AE_MediavsManifestos.html")
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 1), title = "AE in party manifestos and media")
p.vbar(x=list_1, top=list(df_h5_5["manifesto_ae"]), color="black", width=0.3, legend="Manifesto AE", )
p.vbar(x=list_2, top=list(df_h5_5["media_ae"]), color="grey", width=0.3, legend="Media AE")

p.xaxis.ticker = list(df_h5_5.index)
p.xaxis.major_label_overrides = {index:item for index, item in enumerate(df_h5_5["party"])}
p.xaxis.major_label_orientation = "vertical"

show(p)

In [30]:
list_1 = [i-0.15 for i in list(df_h5_5.index)]
list_2 = [i+0.15 for i in list(df_h5_5.index)]

#output_file("Fig11_PC_MediavsManifestos.html")
p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Year', y_axis_label="Share of content", y_range = (0, 1), title = "PC in party manifestos and media")
p.vbar(x=list_1, top=list(df_h5_5["manifesto_pc"]), color="black", width=0.3, legend="Manifesto PC", )
p.vbar(x=list_2, top=list(df_h5_5["media_pc"]), color="grey", width=0.3, legend="Media PC")

p.xaxis.ticker = list(df_h5_5.index)
p.xaxis.major_label_overrides = {index:item for index, item in enumerate(df_h5_5["party"])}
p.xaxis.major_label_orientation = "vertical"

show(p)

In [31]:
df_2020 = df.loc[ (df["manifesto"] == 1 ) & (df["year"]==2020) ]
print(df_2020.shape)
df_2020.head()

(7247, 14)


,_id,author_affiliation,author_name,para_text,site_url,source,title,year,author_affiliation_short,ae,pc,manifesto,year_rec,parliamentary
156022,5f9374ab4307556ebcd05f8d,LaisTeis,LaisTeis,Laisvė ir Teisingumas,nan,party_election_manifesto,nan,2020,Laisve_Teisingumas,False,False,1,2020,1
156023,5f9374ae4307556ebcd05f8e,LaisTeis,LaisTeis,LIETUVOS RESPUBLIKOS SEIMO RINKIMŲ PROGRAMA,nan,party_election_manifesto,nan,2020,Laisve_Teisingumas,True,False,1,2020,1
156024,5f9374b14307556ebcd05f8f,LaisTeis,LaisTeis,"Laisvė ir Teisingumas yra politinė partija, si...",nan,party_election_manifesto,nan,2020,Laisve_Teisingumas,False,True,1,2020,1
156025,5f9374b44307556ebcd05f90,LaisTeis,LaisTeis,Laisvė ir Teisingumas savo politinę veiklą gri...,nan,party_election_manifesto,nan,2020,Laisve_Teisingumas,False,True,1,2020,1
156026,5f9374b84307556ebcd05f91,LaisTeis,LaisTeis,Laisvė ir Teisingumas siekia racionalaus balan...,nan,party_election_manifesto,nan,2020,Laisve_Teisingumas,False,False,1,2020,1


In [32]:
df_2020_gr = pd.DataFrame()
df_2020_gr["Party"] = df_2020.groupby(["author_affiliation_short"])["ae", "pc"].mean().index
df_2020_gr["AE"] = list(df_2020.groupby(["author_affiliation_short"])["ae"].mean())
df_2020_gr["PC"] = list(df_2020.groupby(["author_affiliation_short"])["pc"].mean())
df_2020_gr["ind2"]= list(df_2020_gr.index)

print(df_2020_gr.shape)
df_2020_gr

(17, 4)


<ipython-input-32-22af90421b70>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2020_gr["Party"] = df_2020.groupby(["author_affiliation_short"])["ae", "pc"].mean().index


,Party,AE,PC,ind2
0,DK,0.246445,0.265403,0
1,DP,0.030349,0.118361,1
2,KRIKSCIONIU_SAJUNGA,0.047619,0.535714,2
3,KSS,0.049020,0.222222,3
4,LLP,0.166667,0.444444,4
5,LLRA,0.170732,0.335366,5
6,LP,0.069509,0.121409,6
7,LRLS,0.048565,0.162620,7
8,LSDDP,0.051163,0.144186,8
9,LSDP,0.132931,0.359517,9


In [33]:
#output_file("Fig12_Populist_Manifestos2020Only.html")
tools = "hover"
ind_ae = [int(i)-0.15 for i in list(df_2020_gr["ind2"])]
ind_pc = [int(i)+0.15 for i in list(df_2020_gr["ind2"])]

p = figure(tools=tools, plot_width=1500, plot_height=500, x_axis_label='Parties', y_axis_label="Share of content", y_range = (0, 0.6), title = "Populist Content in Manifestos by Party: 2020")
p.vbar(x=ind_ae, top=list(df_2020_gr["AE"]), color="black", width=0.3, legend="AE", )
p.vbar(x=ind_pc, top=list(df_2020_gr["PC"]), color="grey", width=0.3, legend="PC")

p.xaxis.ticker = list(df_2020_gr["ind2"])
tick_labels = list(df_2020_gr["Party"])
p.xaxis.major_label_overrides = dict(zip(list(df_2020_gr["ind2"]), tick_labels))
p.xaxis.major_label_orientation = 0.785


output_file("Figures.html")
show(p)

# Regression

In [35]:
df = df[df["source"].isin(["party_election_manifesto"])]

In [36]:
df["pop"] = df.apply(lambda x : int(x["ae"]*x["pc"]), axis = 1)
df.head()

<ipython-input-36-24f43b1ab1e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pop"] = df.apply(lambda x : int(x["ae"]*x["pc"]), axis = 1)


,_id,author_affiliation,author_name,para_text,site_url,source,title,year,author_affiliation_short,ae,pc,manifesto,year_rec,parliamentary,pop
133303,5f91c7844307556d58e8ff02,LKDP,LKDP,TARNAUTI LIETUVAI,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1,0
133304,5f91c7874307556d58e8ff03,LKDP,LKDP,LIETUVOS KRIKŠČIONIŲ DEMOKRATŲ PARTIJOS RINKIM...,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1,0
133305,5f91c8224307556ebcd00599,LKDP,LKDP,TARNAUTI LIETUVAI,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1,0
133306,5f91c8254307556ebcd0059a,LKDP,LKDP,LIETUVOS KRIKŠČIONIŲ DEMOKRATŲ PARTIJOS RINKIM...,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1,0
133307,5f91c8274307556ebcd0059b,LKDP,LKDP,2000,nan,party_election_manifesto,nan,2000,LKDP,False,False,1,2000,1,0


In [37]:
dfr = df.groupby(["author_affiliation_short", "site_url", "year"]).agg({"ae" : "mean", "pc":"mean", "pop":"mean", "year_rec":"mean", "manifesto" : "mean", "parliamentary" : "mean"})
print(dfr.shape)
dfr.head()

(100, 6)


ae        pc       pop  \
author_affiliation_short site_url year                                 
CP                                2004  0.062500  0.375000  0.062500   
                                  2008  0.285714  0.142857  0.142857   
DK                       nan      2012  0.261682  0.275701  0.149533   
                                  2016  0.169014  0.302817  0.105634   
                                  2020  0.246445  0.265403  0.151659   

                                        year_rec  manifesto  parliamentary  
author_affiliation_short site_url year                                      
CP                                2004      2004          1              1  
                                  2008      2008          1              0  
DK                       nan      2012      2012          1              0  
                                  2016      2016          1              1  
                                  2020      2020          1              0

In [38]:
dfr["years_after_independence"] = dfr.apply(lambda x : x["year_rec"]-1990, axis = 1)
dfr["first_decade"] = dfr.apply(lambda x : int(x["year_rec"]<2000), axis = 1)
dfr["second_decade"] = dfr.apply(lambda x : int(2000<=x["year_rec"]<2010), axis = 1)
dfr["financial_crisis"] = dfr.apply(lambda x : int(2008<=x["year_rec"]<2011), axis = 1)
dfr.head()

ae        pc       pop  \
author_affiliation_short site_url year                                 
CP                                2004  0.062500  0.375000  0.062500   
                                  2008  0.285714  0.142857  0.142857   
DK                       nan      2012  0.261682  0.275701  0.149533   
                                  2016  0.169014  0.302817  0.105634   
                                  2020  0.246445  0.265403  0.151659   

                                        year_rec  manifesto  parliamentary  \
author_affiliation_short site_url year                                       
CP                                2004      2004          1              1   
                                  2008      2008          1              0   
DK                       nan      2012      2012          1              0   
                                  2016      2016          1              1   
                                  2020      2020          1              0   

                                        years_after_independence  \
author_affiliation_short site_url year                             
CP                                2004                      14.0   
                                  2008                      18.0   
DK                       nan      2012                      22.0   
                                  2016                      26.0   
                                  2020                      30.0   

                                        first_decade  second_decade  \
author_affiliation_short site_url year                                
CP                                2004             0              1   
                                  2008             0              1   
DK                       nan      2012             0              0   
                                  2016             0              0   
                                  2020             0              0   

                                        financial_crisis  
author_affiliation_short site_url year                    
CP                                2004                 0  
                                  2008                 1  
DK                       nan      2012                 0  
                                  2016                 0  
                                  2020                 0

In [39]:
y = dfr["ae"]
X = dfr[["manifesto", "parliamentary", "years_after_independence"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     ae   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     10.38
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           8.22e-05
Time:                        15:53:05   Log-Likelihood:                 58.170
No. Observations:                 100   AIC:                            -110.3
Df Residuals:                      97   BIC:                            -102.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
manifesto               

In [40]:
y = dfr["pc"]
X = dfr[["manifesto", "parliamentary", "years_after_independence"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     pc   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     5.572
Date:                Sun, 21 Apr 2024   Prob (F-statistic):            0.00512
Time:                        15:53:25   Log-Likelihood:                 43.105
No. Observations:                 100   AIC:                            -80.21
Df Residuals:                      97   BIC:                            -72.40
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
manifesto               

In [41]:
y = dfr["ae"]
X = dfr[["manifesto", "parliamentary", "first_decade", "second_decade"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     ae   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.162
Method:                 Least Squares   F-statistic:                     7.395
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           0.000165
Time:                        15:53:51   Log-Likelihood:                 58.868
No. Observations:                 100   AIC:                            -109.7
Df Residuals:                      96   BIC:                            -99.31
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
manifesto         0.2217      0.026      8.400

In [42]:
y = dfr["pc"]
X = dfr[["manifesto", "parliamentary", "first_decade", "second_decade"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     pc   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     3.898
Date:                Sun, 21 Apr 2024   Prob (F-statistic):             0.0112
Time:                        15:54:12   Log-Likelihood:                 43.415
No. Observations:                 100   AIC:                            -78.83
Df Residuals:                      96   BIC:                            -68.41
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
manifesto         0.3093      0.031     10.043

In [43]:
y = dfr["ae"]
X = dfr[["manifesto", "parliamentary", "financial_crisis"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     ae   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     9.255
Date:                Sun, 21 Apr 2024   Prob (F-statistic):           0.000210
Time:                        15:54:36   Log-Likelihood:                 57.205
No. Observations:                 100   AIC:                            -108.4
Df Residuals:                      97   BIC:                            -100.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
manifesto            0.1953      0.021  

In [44]:
y = dfr["pc"]
X = dfr[["manifesto", "parliamentary", "financial_crisis"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     pc   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.612
Date:                Sun, 21 Apr 2024   Prob (F-statistic):              0.205
Time:                        15:54:54   Log-Likelihood:                 39.303
No. Observations:                 100   AIC:                            -72.61
Df Residuals:                      97   BIC:                            -64.79
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
manifesto            0.2590      0.025  

In [45]:
y = dfr["pop"]
X = dfr[["manifesto", "parliamentary", "financial_crisis"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    pop   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     4.419
Date:                Sun, 21 Apr 2024   Prob (F-statistic):             0.0146
Time:                        15:55:17   Log-Likelihood:                 67.936
No. Observations:                 100   AIC:                            -129.9
Df Residuals:                      97   BIC:                            -122.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
manifesto            0.1224      0.019  

In [46]:
y = dfr["pop"]
X = dfr[["manifesto", "parliamentary", "first_decade", "second_decade"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    pop   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     4.268
Date:                Sun, 21 Apr 2024   Prob (F-statistic):            0.00711
Time:                        15:55:41   Log-Likelihood:                 69.836
No. Observations:                 100   AIC:                            -131.7
Df Residuals:                      96   BIC:                            -121.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
manifesto         0.1470      0.024      6.216

In [47]:
y = dfr["pop"]
X = dfr[["manifesto", "parliamentary", "years_after_independence"]]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    pop   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     5.783
Date:                Sun, 21 Apr 2024   Prob (F-statistic):            0.00424
Time:                        15:56:01   Log-Likelihood:                 69.208
No. Observations:                 100   AIC:                            -132.4
Df Residuals:                      97   BIC:                            -124.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
manifesto               